In [ ]:
from pyscf import gto, scf, fci, lo
import netket as nk; import netket.experimental as nkx
import numpy as np
import itertools
import qiskit
from qiskit.quantum_info import Pauli, SparsePauliOp
from collections import defaultdict
import tensorflow as tf
import tensorflow as tf
import sys
import os
import pandas as pd
import itertools
from itertools import product
import matplotlib.pyplot as plt
#sys.path.append(os.path.dirname(os.getcwd()) + "/SAMPLER_LOCAL_IMPORT")
#from Sampling_Quantum import *
#from New_MCMC_Proposal import *
sys.path.append(os.getcwd() + "/Code_download_Bell_2")
from MCMC_funs_Leyden import *

print(qiskit.version.get_version_info())
os.getcwd()

In [ ]:
def all_configs(n):
    return [np.array(v) for v in product([-1, 1], repeat=n)]



def Sampling_MCMC_trajectories(problem_inst, Transition_matrix, sample_size=10000, 
                               burn=1000, method='Quantum', init_config=None):

    n = problem_inst.n
    beta = problem_inst.T
    prob_dist = np.zeros(2**n)

    #exact_dist = np.exp(-beta * Proposal_object.Energy_array)
    #exact_dist = exact_dist / np.sum(exact_dist)

    #err_hist = []
    key_list = []

    if init_config==None:
        s = np.random.choice([1,-1],size=n)
    else: s = init_config

    int_key = spinconf2int(s)
    #print(int_key)
    
    for k in range(burn):
        #s = Proposal_object.generate_MCMC_trajectories(Transition_matrix, s)
         int_key = generate_move(transition_mat=Transition_matrix, state=int_key)

    for k in range(sample_size):
        #s = Proposal_object.generate_MCMC_trajectories(Transition_matrix, s)
        int_key = generate_move(transition_mat=Transition_matrix, state=int_key)
        #key = spinconf2int(s)
        prob_dist[int_key] +=1
        key_list.append(int_key)

    
    return np.flip(prob_dist/np.sum(prob_dist)), key_list




def grad_U(x, J, h, alpha):
    v = np.tanh(alpha * x)
    sech_sq = 1.0 * (1 - v ** 2)
    return alpha * sech_sq * (h + 2 * J @ v)


def adaptive_MALA_step_new(x, cov, epsilon, problem_inst, alpha=3.0):
    J_Q = problem_inst.J_quantum
    h_Q = problem_inst.h_quantum
    cov = cov + 1e-6 * np.eye(len(x))

    mean_shift = x + epsilon * problem_inst.T * cov @ grad_U(x, J_Q, h_Q, alpha)
    noise = np.random.multivariate_normal(np.zeros(len(x)), epsilon**2 * cov)
    x_new = mean_shift + noise

    v_old = np.sign(np.tanh(alpha * x))
    v_new = np.sign(np.tanh(alpha * x_new))
    E_old = problem_inst.E_arr[::-1][spinconf2int(v_old)]
    E_new = problem_inst.E_arr[::-1][spinconf2int(v_new)]

    log_q_forward = -0.5 * noise.T @ np.linalg.inv(epsilon**2 * cov) @ noise
    noise_rev = x - x_new + epsilon * cov @ grad_U(x_new, J_Q, h_Q, alpha)
    log_q_reverse = -0.5 * noise_rev.T @ np.linalg.inv(epsilon**2 * cov) @ noise_rev

    log_accept_ratio = -problem_inst.T * (E_new - E_old) + (log_q_reverse - log_q_forward)
    accept_prob = min(1.0, np.exp(log_accept_ratio))

    if np.random.rand() < accept_prob:
        return x_new, v_new
    else:
        return x, v_old


def adaptive_MALA_T_matrix(problem_inst, alpha=3.0, epsilon=0.3, num_samples=500):
    from tqdm import tqdm

    n = problem_inst.n
    #beta = problem_inst.T

    configs = all_configs(n)
    idx_map = {tuple(c): i for i, c in enumerate(configs)}
    T = np.zeros((2**n, 2**n))

    for v in tqdm(configs, desc="Building T matrix"):
        x = np.arctanh(np.clip(v, -0.999, 0.999)) / alpha
        cov = np.eye(n)

        counts = defaultdict(int)

        v_samples = []
        adapt_window = 20

        for step in range(num_samples):
            #x, v_new = adaptive_MALA_step(x, cov, epsilon, problem_inst, alpha=3.0)
            x, v_new = adaptive_MALA_step_new(x, cov, epsilon, problem_inst, alpha=3.0)
            v_samples.append(v_new)

            key = tuple(v_new.astype(int))
            counts[key] = counts.get(key, 0) + 1

            if (step + 1) % adapt_window == 0 and step > 0:
                if len(v_samples) >= 50:
                    x_hist = np.array([np.arctanh(np.clip(np.tanh(alpha * x0), -0.999, 0.999)) / alpha for x0 in v_samples[-50:]])

                else:
                    x_hist = np.array([np.arctanh(np.clip(np.tanh(alpha * x0), -0.999, 0.999)) / alpha for x0 in v_samples])

                cov = np.cov(x_hist.T)

        i = idx_map[tuple(v)]
        total = sum(counts.values())

        for v_prime, c in counts.items():
            j = idx_map[v_prime]

            if j != i :
                T[j, i] = c / total

        T[i,i] = 1-sum(T[:,i])

    T = normalize_transition_matrix(T, eps=1e-12, verbose=True)

    return T

In [ ]:
def instance_result_reader_writer(filepath, filename_instances):
    from tqdm import tqdm
    instances = pd.read_pickle(filepath + filename_instances)
    tot_instances = np.max(instances['Instance Number'].values)
    print(tot_instances)
    tot_instances = 10
    results = pd.DataFrame(columns = [
        'Instance Number',
        'Spins', 
        'Temperature', 
        'Connectivity',
        'Proposal',
        'Acceptance',
#    'Mismatched',
        'Gap',
        'Gap Lazy'
           ])
    T_logmesh = np.logspace(-3, 3, 50)
    T_round = np.sort( np.kron(np.array([1,5]), 10.**np.arange(-3,4)))
    T_lim = np.array([0, np.inf]) 
    T_arr = np.unique( np.concatenate((T_logmesh, T_round, T_lim))) 
    #T_arr = np.delete(T_arr, -2) 
    T_arr = [1]
    delta_step = 0.2

    for instance_num in range(1, tot_instances+1):
    
        cond  = (instances['Instance Number']==instance_num)
        n            = instances[cond]['Spins'].values[0]
        connectivity = instances[cond]['Connectivity'].values[0]
        J            = instances[cond]['J'].values[0]
        h            = instances[cond]['h'].values[0]
        #J_mismatch   = instances[cond]['J Mismatch'].values[0]
        #h_mismatch   = instances[cond]['h Mismatch'].values[0]

        print('Starting problem instance', instance_num, 'of', tot_instances, 'with n =', n)

        problem_inst = ProblemInstance(J, h)
        #problem_inst_mismatch = ProblemInstance(J_mismatch, h_mismatch)

        proposal_mats = {}
        proposal_mats['local'] = local_proposal_mat(n)
        proposal_mats['uniform']= uniform_proposal_mat(n)
        proposal_mats['Haar']= Haar_random_proposal_mat(n)
        proposal_mats['quantum_avg'] = quantum_proposal_mat_avg(problem_inst)
        #proposal_mats['quantum_time_mid_gamma_mid'] = quantum_proposal_time_homogeneous(problem_inst, t_val="t_mid")
        proposal_mats['quantum_time_mid_gamma_mid_Trotter'] = quantum_proposal_time_homogeneous_Trotter_circuit(problem_inst, delta_step, t_val="t_mid")
        
        #proposal_mats['quantum_time_uplim_gamma_mid'] = quantum_proposal_time_homogeneous(problem_inst, t_val="t-uplim")
        #proposal_mats['quantum_time_lowlim_gamma_mid'] = quantum_proposal_time_homogeneous(problem_inst, t_val="t-llim")
    
        #norm_diff_trotter_exact_proposal = np.linalg.norm(proposal_mats['quantum_time_mid_gamma_mid_Trotter']-proposal_mats['quantum_time_mid_gamma_mid'])
        #norm_exact_proposal = np.linalg.norm(proposal_mats['quantum_time_mid_gamma_mid'])

        for T in T_arr:
            problem_inst.T = T
            #problem_inst_mismatch.T = T 
            #proposal_mats['Wolff_cluster_update'] = Wolff_cluster_Proposal_matrix(problem_inst)

            #for prop_type, accept_type in product(['local', 'uniform', 'quantum'], ['metropolis', 'glauber']):
            #for prop_type, accept_type in product(['local', 'uniform', 'quantum_avg', 'Haar', 
            #                                   'quantum_time_mid_gamma_mid_Trotter', 'Wolff_cluster_update', 'Exchange_cluster',
            #                                    "Continuous-HMC", "Random_walk_MC", "MALA_MC"], ['metropolis']):    #

            #for prop_type, accept_type in product(["Random_walk_MC"], ['metropolis']):    # 
            for prop_type, accept_type in product(["MALA_MC"], ['metropolis']):     


                if prop_type == "Continuous-HMC":
                    #transition_mat = HMC_T_matrix(problem_inst, epsilon=0.2, L=10, alpha=3.0, num_samples=500, num_p_samples=30)
                    transition_mat = HMC_T_matrix_vectorized(problem_inst, epsilon=0.2, L=10, alpha=3.0,
                            num_samples=800, num_p_samples=40)

                elif prop_type == "Random_walk_MC":
                    transition_mat = RWM_T_matrix(problem_inst, alpha=3.0, epsilon=0.2, num_samples=1000)
                    print(transition_mat, np.linalg.eigvals(transition_mat))
                #elif prop_type == "MALA_MC":
                #    transition_mat = adaptive_MALA_T_matrix(problem_inst, alpha=3.0, epsilon=0.2, num_samples=500)

                elif prop_type == "MALA_MC":
                    #pass
                    #transition_mat = build_transition_matrix_mala(problem_inst, 
                    #                                              epsilon=0.2, alpha=3.0, num_samples=500, adapt_every=10)
                    transition_mat = adaptive_MALA_T_matrix(problem_inst, alpha=3.0, epsilon=0.2, num_samples=1000)
                    #print(transition_mat, np.linalg.eigvals(transition_mat))

                elif prop_type == 'Exchange_cluster':
                    transition_mat = Exchange_cluster_transition_matrix(problem_inst, num_samples=2000)

                else:
                    transition_mat = make_transition_mat(problem_inst, proposal_mats[prop_type], acceptance=accept_type)
                    
                
                gap, gap_lazy = abs_spectral_gap(transition_mat)
                print(gap)

                results_datum = {
                    'Instance Number': instance_num,
                    'Spins': n, 
                    'Temperature': T, 
                    'Connectivity': connectivity,
                    'Proposal': prop_type,
                    'Acceptance': accept_type,
                    'Mismatched': False,
                    'Gap': gap,
                    'Gap Lazy': gap_lazy,
                    "delta_time_step_Trotter": delta_step
                }


               results = pd.concat([results,pd.DataFrame([results_datum])], ignore_index=True)


filepath = '/Users/msajjan/Desktop/PROJECTS/RBM_phase_project/PLOTS/CONVERGENCE_COMPARISON_DATA/250_instances_run_Jul_1st/LOCAL_MACHINE_IMPORT'
filename_instances =  '/instances_new.pkl'

instance_result_reader_writer(filepath=filepath, filename_instances=filename_instances)